In [1]:
from dataset import (
    BuildingDataset,
    show_segmentation,
    data_loaders,
    show_segmentation_v2,
    MiyazakiDataset,
    get_simple_transform,
    get_transform,
)
from model import (
    ModelConfig,
    new_model,
    TwoMLPRegression,
    EnhancedTwoMLPRegression,
    SimpleBuildingHeightPred,
    new_pretrained_model,
)
from dataset import NUMBER_OF_CLASSES
import dataset
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
import torchvision.transforms as T
import torch
import logging
from model import train, load_fine_tuned_model, show_prediction_results
from torch.nn import SmoothL1Loss

logging.getLogger("matplotlib").setLevel(logging.WARNING)

In [ ]:
def train_pretrained_model(model_name: str):
    model = new_pretrained_model(
        new_model_name=model_name,
        pretrained_checkpoint_path="checkpoints/pretrained_seg_miyazaki_epoch_9.pt",
        height_pred_class=EnhancedTwoMLPRegression,
        height_pred_loss_fn=SmoothL1Loss(),
        height_pred_after_roi_pool=True,
        sample_equal=False,
        map_location="cpu",
    )

    data_loader, data_loader_test = data_loaders(
        "datasets/images_annotated",
        dataset_cls=BuildingDataset,
        train_batch_size=2,
        test_batch_size=1,
        test_split=0.99,
        num_workers=4,
    )

    epoch_losses, epoch_loss_dicts = train(
        model,
        data_loader,
        data_loader_test,
        num_epochs=2,
        eval_iterations=0,
        checkpoint_dir="checkpoints",
        checkpoint_prune_threshold=3,
    )

    return model, epoch_losses, epoch_loss_dicts


model, epoch_losses, epoch_loss_dicts = train_pretrained_model("test_v2")

In [ ]:
if "data_loader_test_inf" not in locals():
    model = load_fine_tuned_model("checkpoints/pretrained_model_v1_epoch_0.pt")

    data_loader_inf, data_loader_test_inf = data_loaders(
        "datasets/images_annotated",
        dataset_cls=BuildingDataset,
        transform=get_transform,
        train_batch_size=2,
        test_batch_size=1,
        test_split=0.2,
        num_workers=4,
    )

inf_images, inf_targets = next(iter(data_loader_test_inf))

predictions = show_prediction_results(
    model, inf_images, inf_targets, mask_threshold=0.7, box_score_threshold=0.7
)

logging.info(f"Prediction keys: {predictions[0].keys()}")

In [ ]:
model_cfg = ModelConfig(
    name="inf_model",
    num_classes=NUMBER_OF_CLASSES,
    mask_hidden_layer_size=256,
    trainable_backbone_layers=3,
)

model = new_model(model_cfg)

checkpoint_path = "checkpoints/pretrained_seg_miyazaki_epoch_9.pt"
checkpoint_dict = torch.load(checkpoint_path, map_location="cpu")
model.load_state_dict(checkpoint_dict["model_state_dict"], strict=True)

In [ ]:
data_loader, data_loader_test = data_loaders(
    "datasets/miyazaki",
    dataset_cls=MiyazakiDataset,
    get_transform=get_simple_transform,
    train_batch_size=13,
    test_batch_size=3,
    test_split=0.01,
    num_workers=8,
)
data_loader_test_inf_iter = iter(data_loader_test)

In [ ]:
og_data_loader, og_data_loader_test = data_loaders(
    "datasets/images_annotated/",
    dataset_cls=BuildingDataset,
    get_transform=get_simple_transform,
    train_batch_size=13,
    test_batch_size=3,
    test_split=0.01,
    num_workers=8,
)
og_data_loader_test_inf_iter = iter(og_data_loader_test)

In [ ]:
dataset = BuildingDataset(
    "datasets/images_annotated/",
    transforms=get_simple_transform(train=True),
)
dataset[1993][1]["image_name"]

In [ ]:
# inf_images, inf_targets = next(data_loader_test_inf_iter)
# inf_images, inf_targets = next(og_data_loader_test_inf_iter)
# inf_images, inf_targets = dataset[1321]
# inf_images = [inf_images]
# inf_targets = [inf_targets]
# inf_images = inf_images.unsqueeze(0)


predictions = show_prediction_results(
    model, inf_images, inf_targets, mask_threshold=0.6, box_score_threshold=0.6
)

logging.info(f"Prediction keys: {predictions[0].keys()}")

In [2]:
def train_pretrained_model(model_name: str):
    model = new_pretrained_model(
        new_model_name=model_name,
        pretrained_checkpoint_path="checkpoints/pretrained_seg_miyazaki_epoch_9.pt",
        height_pred_class=EnhancedTwoMLPRegression,
        height_pred_loss_fn=SmoothL1Loss(),
        height_pred_after_roi_pool=False,
        sample_equal=False,
        map_location="cpu",
    )

    data_loader, data_loader_test = data_loaders(
        "datasets/images_annotated",
        dataset_cls=BuildingDataset,
        train_batch_size=2,
        test_batch_size=1,
        test_split=0.05,
        num_workers=4,
    )

    return train(
        model,
        data_loader,
        data_loader_test,
        num_epochs=3,
        eval_iterations=0,
        checkpoint_dir="checkpoints",
        checkpoint_prune_threshold=3,
    )


model, epoch_losses, epoch_loss_dicts = train_pretrained_model(
    "fine_tuned_model_miyazaki_9_v2"
)

INFO:root:Size of training set 2086. Sise of test set: 109
INFO:root:No checkpoint found, starting from scratch


Epoch: [0] / [2]  [   0/1043]  eta: 2:41:00  lr: 0.000010  loss: 2.0638 (2.0638)  loss_classifier: 0.2429 (0.2429)  loss_box_reg: 0.2540 (0.2540)  loss_building_height_reg: 0.7377 (0.7377)  loss_mask: 0.6372 (0.6372)  loss_objectness: 0.1597 (0.1597)  loss_rpn_box_reg: 0.0323 (0.0323)  time: 9.2627  data: 3.7527
Epoch: [0] / [2]  [  10/1043]  eta: 2:12:58  lr: 0.000060  loss: 4.1700 (4.2919)  loss_classifier: 0.3990 (0.3725)  loss_box_reg: 0.4952 (0.4636)  loss_building_height_reg: 2.1011 (2.4646)  loss_mask: 0.5999 (0.5942)  loss_objectness: 0.2562 (0.3042)  loss_rpn_box_reg: 0.0656 (0.0928)  time: 7.7237  data: 0.3420
Epoch: [0] / [2]  [  20/1043]  eta: 2:18:13  lr: 0.000110  loss: 4.5701 (4.8540)  loss_classifier: 0.4130 (0.3964)  loss_box_reg: 0.4980 (0.5098)  loss_building_height_reg: 2.2933 (2.8347)  loss_mask: 0.5979 (0.5921)  loss_objectness: 0.3620 (0.3862)  loss_rpn_box_reg: 0.1273 (0.1348)  time: 8.0495  data: 0.0010
Epoch: [0] / [2]  [  30/1043]  eta: 2:13:55  lr: 0.000160 